In [1]:
import cv2
import os
import numpy as  np
import matplotlib.pyplot as plt

In [2]:
### File directory
datadir_face_test = 'testingData/face/'
datadir_bg_test = 'testingData/background/'
datadir_face_train = 'trainingData/face/'
datadir_bg_train = 'trainingData/background/'
trainingFaceImages = os.listdir(datadir_face_train)
trainingBgImages = os.listdir(datadir_bg_train)
testingFaceImages = os.listdir(datadir_face_test)
testingBgImages = os.listdir(datadir_bg_test)
### Parameters
nDim = 900 

In [3]:
### Training face process 
U_face = np.zeros((nDim, 1)) # mean
sigma_face = np.zeros((nDim, nDim)) # variance of face
diag_sigma_face = np.zeros((nDim, nDim)) # covariance of face
x_training_face = [] # x array

for iFile in trainingFaceImages:
    ### Load the training image
    origIm = cv2.imread(datadir_face_train + iFile)
      
    ### each image x codes:
    # sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
    # sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)
    # imgeGradient = sobelx + sobely
    # x_training_face.append(np.reshape(imgeGradient, (-1,1)))
    x_training_face.append(np.reshape(origIm, (-1,1)))

xNp_training_face = np.array(x_training_face) 
### MLE mean
U_face = np.mean(xNp_training_face, axis = 0)

#print(f" U_face[100] = {U_face[100]}")

### MLE variance
for x in xNp_training_face:
    sigma_face = sigma_face + (x - U_face) * np.transpose(x-U_face)
sigma_face = sigma_face / len(x_training_face)    

#print(f" diag_sigma[100] = {np.diag(sigma_face)[100]}")

### MLE covaricance
diag_sigma_face = np.diag(np.diag(sigma_face))

#print(f"xNp.shape = {xNp_training_face.shape}")
#print(len(x_training_face))
#print(U_face.shape)
#print(sigma_face.shape)
#print(diag_sigma_face.shape)

In [4]:
### Training background process
U_bg = np.zeros((nDim, 1)) # mean
sigma_bg = np.zeros((nDim, nDim)) # variance of bg
diag_sigma_bg = np.zeros((nDim, nDim)) # covariance of bg
x_training_bg = [] # x array

for iFile in trainingBgImages:
    ### Load the training image
    origIm = cv2.imread(datadir_bg_train + iFile)
      
    ### each image x codes:
    x_training_bg.append(np.reshape(origIm, (-1,1)))

xNp_training_bg = np.array(x_training_bg)
### MLE mean
U_bg = np.mean(xNp_training_bg, axis = 0)

### MLE variance
for x in xNp_training_bg:
    sigma_bg = sigma_bg + (x - U_bg) * np.transpose(x-U_bg)
sigma_bg = sigma_bg / len(x_training_bg)

### MLE covaricance
diag_sigma_bg = np.diag(np.diag(sigma_bg))

#print(f"xNp_bg.shape = {xNp_training_bg.shape}")
#print(len(x_training_bg))
#print(U_bg.shape)
#print(sigma_bg.shape)
#print(diag_sigma_bg.shape)

In [5]:
### Testing face images
x_testing_face = [] # x array
N_classified_face_from_face = 0
accuracy_face = 0

for iFile in testingFaceImages:  
    ### Load the testing image 
    origIm = cv2.imread(datadir_face_test + iFile)
    
    ### each image x codes:
    x_testing_face.append(np.reshape(origIm, (-1,1)))

for testX in x_testing_face:
    cov_face = np.reshape(np.diag(diag_sigma_face), (-1, 1))
    term1_face = np.sum(np.log(cov_face))
    term2_face = np.sum(np.power((testX - U_face),2) / (2 * cov_face))
    probFace = -0.5 * term1_face - term2_face
    
    cov_bg = np.reshape(np.diag(diag_sigma_bg), (-1, 1))
    term1_bg = np.sum(np.log(cov_bg))
    term2_bg = np.sum(np.power((testX - U_bg),2) / (2 * cov_bg))
    probBg = -0.5 * term1_bg - term2_bg
    
    if probFace > probBg:
        N_classified_face_from_face += 1
    
print(N_classified_face_from_face)
print(len(x_testing_face))
accuracy_face = N_classified_face_from_face / len(x_testing_face)
print(accuracy_face)

191
232
0.8232758620689655


In [6]:
### Testing background images
x_testing_bg = [] # x array
N_classified_bg_from_bg = 0
accuracy_bg = 0

for iFile in testingBgImages:  
    ### Load the testing image 
    origIm = cv2.imread(datadir_bg_test + iFile)
    
    ### each image x codes:
    x_testing_bg.append(np.reshape(origIm, (-1,1)))

for testX in x_testing_bg:
    cov_face = np.reshape(np.diag(diag_sigma_face), (-1, 1))
    term1_face = np.sum(np.log(cov_face))
    term2_face = np.sum(np.power((testX - U_face),2) / (2 * cov_face))
    probFace = -0.5 * term1_face - term2_face
    #print(probFace)
    
    cov_bg = np.reshape(np.diag(diag_sigma_bg), (-1, 1))
    term1_bg = np.sum(np.log(cov_bg))
    term2_bg = np.sum(np.power((testX - U_bg),2) / (2 * cov_bg))
    probBg = -0.5 * term1_bg - term2_bg
    #print(probBg)
    
    if probFace < probBg:
        N_classified_bg_from_bg += 1
    
print(N_classified_bg_from_bg)
print(len(x_testing_bg))
accuracy_bg = N_classified_bg_from_bg / len(x_testing_bg)
print(accuracy_bg)

421
564
0.7464539007092199
